In [1]:
import pandas as pd
import numpy as np

In [2]:
N = 100000000

In [3]:
HDF_KWARGS = {
    'data_columns': True,  # 便于列查询
    'append': True,  # 数据可添加
    'complevel': 9,
    'complib': 'blosc:blosclz', # 默认库。速度最快
}

In [4]:
codes = [str(i).zfill(6) for i in range(1,1000)]

In [5]:
df = pd.DataFrame({
    'date':
    pd.date_range('1990-11-01 00:00', periods=N, freq='s'),
    'stock_code':
    np.random.choice(codes, N),
    'pct':
    np.random.random(N),
    'price':
    np.random.random(N),
})

In [6]:
df.set_index(['date', 'stock_code'], inplace=True)

In [7]:
df.tail()

,,pct,price
date,stock_code,,
1994-01-01 09:46:35,000509,0.068158,0.263386
1994-01-01 09:46:36,000944,0.575605,0.875603
1994-01-01 09:46:37,000220,0.137698,0.601171
1994-01-01 09:46:38,000339,0.948101,0.681946
1994-01-01 09:46:39,000451,0.658560,0.160573


# 写入

In [8]:
# 带索引写入 7min
# 不带索引 ?
store = pd.HDFStore('large.h5')
store.append("df", df, index=False, **HDF_KWARGS)

C:\Users\liude\anaconda3\envs\zipline\lib\site-packages\tables\leaf.py:353: RuntimeWarning: overflow encountered in long_scalars
  expected_mb = (expectedrows * rowsize) // MB


## 创建索引

In [10]:
# 创建索引 5min35s
store.create_table_index('df', optlevel=9, kind='full')

# 查询

## 主坐标

In [11]:
# 15ms
store.select("df", "date>=pd.Timestamp('1994-01-01 09:46:35') and date<=pd.Timestamp('1994-01-01 12:46:35')")

,,pct,price
date,stock_code,,
1994-01-01 09:46:35,000509,0.068158,0.263386
1994-01-01 09:46:36,000944,0.575605,0.875603
1994-01-01 09:46:37,000220,0.137698,0.601171
1994-01-01 09:46:38,000339,0.948101,0.681946
1994-01-01 09:46:39,000451,0.658560,0.160573


## 次坐标

In [12]:
# 7.5s 20w rows
store.select("df","stock_code = ['000001','000099']")

,,pct,price
date,stock_code,,
1990-11-01 00:02:22,000099,0.428784,0.803352
1990-11-01 00:18:48,000099,0.154121,0.991506
1990-11-01 00:23:23,000099,0.471175,0.242316
1990-11-01 00:36:36,000001,0.416093,0.110704
1990-11-01 00:45:48,000001,0.937726,0.691815
...,...,...,...
1994-01-01 09:10:00,000001,0.371614,0.840402
1994-01-01 09:18:09,000099,0.551704,0.096184
1994-01-01 09:18:44,000001,0.950649,0.580698


In [13]:
# 44s 2000w rows
store.select("df","price > 0.8")

,,pct,price
date,stock_code,,
1990-11-01 00:00:15,000846,0.446632,0.966218
1990-11-01 00:00:24,000592,0.552473,0.955104
1990-11-01 00:00:27,000478,0.541749,0.892235
1990-11-01 00:00:29,000092,0.682438,0.949997
1990-11-01 00:00:43,000344,0.675081,0.805278
...,...,...,...
1994-01-01 09:46:17,000807,0.354347,0.823920
1994-01-01 09:46:18,000778,0.826694,0.922385
1994-01-01 09:46:19,000208,0.553176,0.953628


## 联合

In [14]:
# 15s 630w rows
store.select("df", "date >= pd.Timestamp('1993-01-01 09:46:35') and price > 0.8")

,,pct,price
date,stock_code,,
1993-01-01 09:46:53,000152,0.931940,0.961561
1993-01-01 09:46:58,000907,0.838805,0.952002
1993-01-01 09:47:01,000918,0.700786,0.843897
1993-01-01 09:47:06,000390,0.770505,0.850942
1993-01-01 09:47:08,000525,0.279420,0.982480
...,...,...,...
1994-01-01 09:46:17,000807,0.354347,0.823920
1994-01-01 09:46:18,000778,0.826694,0.922385
1994-01-01 09:46:19,000208,0.553176,0.953628
